# Pulling AWR data into Python

This notebook uses the pyawr module to bring data into Python (Pandas dataframes).  It also describes some of the utility functions defined in pyawr.

In [2]:
import os
import pandas as pd
from pyawr import connect, AwrMeas, as_list, meas_from_graph, open_example

In [3]:
# connect to AWRDE
awrde, awrc = connect()  # awrde is the pointed to the awrde COM object while awrc is the AWR defined constants

***** Loading type information for Microwave Office  ********************


In [4]:
# Open the lumped filter example
open_example(awrde, 'LPF_lumped.emp')

In [5]:
# Let's see what graphs it has
[x.Name for x in awrde.Project.Graphs]

['Passband and Stopband', 'Input Match', 'Group Delay', 'Output Match']

In [6]:
# We need to simulate before we can get results
awrde.Project.Simulate()

In [7]:
# Get all the measurements on the Passband and Stopband graph
measurements = as_list(awrde.Project.Graphs('Passband and Stopband').Measurements)
measurements

[<win32com.gen_py.5AE2215C-270C-470B-A2B6-609A964E53A2x0x13x0.IMeasurement instance at 0x2757661343872>,
 <win32com.gen_py.5AE2215C-270C-470B-A2B6-609A964E53A2x0x13x0.IMeasurement instance at 0x2757661299152>,
 <win32com.gen_py.5AE2215C-270C-470B-A2B6-609A964E53A2x0x13x0.IMeasurement instance at 0x2757660324808>]

## Transfer Measurement to DataFrame

Since it is common to want to transfer data from a measurement to a DataFrame so that we can work with it in Python we have created a class to simplify this.

### Unswept Data

In [8]:
m1 = AwrMeas(measurements[0])  # it takes a measurement object as an argument
print(m1)

AwrMeas(LPF:DB(|S(1,1)|),type=mwMDT_ReflectionData,dim=2,pts=95)


In [9]:
# the AWR measurement has several useful properties
print(m1.name)
print(m1.source)
print(m1.type)
print(m1.plot_dim)
print(m1.x_units)
print(m1.y_units)

DB(|S(1,1)|)
LPF
mwMDT_ReflectionData
2
mwUT_Frequency
mwUT_None


In [10]:
# as well as the data
m1.df.head()

,name,source,x,y
0,"DB(|S(1,1)|)",LPF,100000000.0,-17.874667
1,"DB(|S(1,1)|)",LPF,110000000.0,-17.757855
2,"DB(|S(1,1)|)",LPF,120000000.0,-17.773001
3,"DB(|S(1,1)|)",LPF,130000000.0,-17.920683
4,"DB(|S(1,1)|)",LPF,140000000.0,-18.203183


### Swept Data

In the case of swept data we must also bring over the sweep variables.  This is done automatically in AwrMeas and those variables are added to the dataframe.

Let's look at an 

In [15]:
open_example(awrde, 'BJT_Characterization.emp')
ms = as_list(awrde.Project.Graphs('Dynamic Load Line').Measurements)

In [35]:
swept_meas = AwrMeas(ms[0])

In [36]:
swept_meas.df.head()

,Istep,Istep_unit,name,source,x,y
0,0.0,mwUT_Current,IVCurve(),IV.AP_DC,0.0,3.297266e-23
1,0.0,mwUT_Current,IVCurve(),IV.AP_DC,0.1,1.534091e-12
2,0.0,mwUT_Current,IVCurve(),IV.AP_DC,0.2,3.101040e-12
3,0.0,mwUT_Current,IVCurve(),IV.AP_DC,0.3,4.779475e-12
4,0.0,mwUT_Current,IVCurve(),IV.AP_DC,0.4,6.579827e-12


Here we see that this dataframe has additional columns for Istep, the swept variable, and Istep_unit, the units for the Istep variable.